<a href="https://colab.research.google.com/github/Rocks-n-Code/PythonCourse/blob/master/7%20-%20UnSupML_LogFacies%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Log Facies with Unsupervised Machine Learning
---
## K-Means Clustering

##### _A [recording](https://youtu.be/_DTN5ZhjJCY?t=4930) of a walkthrough of this notebook is avalible via AAPG Energy Minerals Division's Freeware for Freelancers lecture._
---
K-Means clustering is a commonly used method to assign points to a cluster. This is done by minimizing the Euclidean distance to the centeroid of each cluster. K, or the number of clusters, is defined by the user. For our example we'll optimize the number of clusters by minimizing the (squared) diffrence between from each cluster's centeroid, or within-sum-of-squares (WSS), and the similarity to a cluster with dismilarity to neighboring clusters with a silouette score. The silhouette score has a range from -1 to 1 with an optimum score being 1.

In [ ]:
pip install lasio

In [ ]:
import pandas as pd
import lasio
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import euclidean_distances

%matplotlib inline
pd.options.display.max_columns = 999

Load in our data from an LAS log and fix some bad values.

In [ ]:
las = lasio.read("https://raw.githubusercontent.com/Rocks-n-Code/PythonCourse/master/data/4210932323_ENCANA_DELAWARE_61_T1_16_1H_LAS.las")
las_df = las.df()

#Getting rid of some bad values
las_df = las_df.replace(-936.7891,np.NaN)
las_df = las_df.replace(-624.4871,np.NaN)
las_df = las_df.replace(-187.2639,np.NaN)

#Preview our data
las_df.loc[2000:2002]

For the purpose of demonstration let's only use some of the information. Once we seperate out 1000' to work with let's reduce the influence of different units.

In [ ]:
#Pulling a subset to work with
X = las_df.loc[2000:3000,['GR','MSFL','LLS','LLD','DPHI','NPHI','PE']]

#Dimensionality Reduction
scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X), index=X.index, columns=X.columns)
X.head()

Notice that we've made a copy of the LAS dataframe and that the units are now between 0-1.

---

## Silhoutte Score

We'll now do some testing with the Silhoutte Score to try and find an appropriate number of log facies.

In [ ]:
k = list(range(2,13))
score=[]

for n_cluster in k:
    km = KMeans(n_clusters=n_cluster).fit(X)
    silhouette_avg = silhouette_score(X, km.labels_)
    score.append(silhouette_score(X, km.labels_))

plt.plot(k, score, 'o-')
plt.xlabel("Clusters (k)")
plt.ylabel("Silhouette Score")
plt.show()

Note the high peak at k=4.

---

##  WSS & Elbow Method

In [ ]:
inertias = []
for i in k:
    km = KMeans(n_clusters=i)
    km.fit(X)
    inertias.append(km.inertia_)
plt.plot(k, inertias)
plt.xlabel("Clusters")
plt.ylabel("Within Sum of Square")
plt.show()

We are looking for where the "elbow" is in the WSS, or approximately 5.

---

## K-Means

Now that we've chosen a number of clusters to use we can fit our model and display our data.

In [ ]:
km = KMeans( n_clusters=4 )
km.fit(X)

Let's also calculate the euclidean distances between each point and their respective cluster centeroid.  This will give us a log to display to measure the fit of each depth to it's cluster.

In [ ]:
#Write the cluster labels to a column in the df
labels = km.labels_
X['FACIES'] = labels

clsts = list(set(labels.astype(np.float64)))
log_cols = ['GR','MSFL','LLS','LLD','DPHI','NPHI','PE']
x = X.shape[0]
X['euc_dis'] = np.nan
for i, clst in enumerate(clsts):

    #Center of cluster i
    centers = [km.cluster_centers_[i]]

    #Calculate distance of all points to center i; write to temp col in df
    X['temp'] = euclidean_distances(X[log_cols].values, centers)

    #Place those euclidean distances where the df's col FACIES is == to i
    X['euc_dis'] = X['euc_dis'].where(X['FACIES'] != i, other=X['temp'])

#Drop temp column
X.drop(columns=['temp'],inplace=True)

X.head()

While the computer works better with scaled log values we are use to reading well log values. Let's write the original (Un-Reduced) log values back to X for our plot.

In [ ]:
X[['GR','MSFL','LLS','LLD','DPHI','NPHI','PE']] = las_df.loc[2000:3000,['GR','MSFL','LLS','LLD','DPHI','NPHI','PE']]
X.head()

To make it easier to plot these logs I've included some original and some tweaked functions for our purpose.

In [ ]:
def rect(ax,x,y,w,h,c):
    #Make rectangle with plt.Rectangle((x,y@ Lower left),width,height,color)
    polygon = plt.Rectangle((x,y),w,h,color=c)
    #Add it to axis
    ax.add_patch(polygon)

def GR_fill(ax,X,Y):
    plt.plot(X,Y,lw=0)

    #Make a custom color ramp
    cmap = mpl.colors.LinearSegmentedColormap.from_list("", ["yellow","yellow",'goldenrod','darkgoldenrod',"dimgrey",'black','black'])

    #Step distance
    dy = Y[1]-Y[0]

    #Max GR value
    N  = float(X.max())

    #Make rectangles with color based on GR values
    for n, (x,y) in enumerate(zip(X,Y)):
        color = cmap(x/N)
        rect(ax,x,y,N-x,dy,color)

def log_plot(X, GR, MSFL, LLS, LLD, DPHI, NPHI, PE, FACIES, EucD):
    '''
    X: df, Dataframe of log values and cluster output.
    GR,MSFL,LLS,LLD,DPHI,NPHI,PE,FACIES,EucD: str, Col names

    This is a tweaked version of Brendon Hall's log plot.  If you haven't read the article you really should:
    https://csegrecorder.com/articles/view/geochemical-facies-analysis-using-unsupervised-machine-learning
    Brilliant guy and good human being. Stop by ENTHOUGHT's booth next time you're at a conference and say hi!
    '''
    cluster=np.repeat(np.expand_dims(X[FACIES].values,1), 100, 1)

    ztop=X.index.min(); zbot=X.index.max()
    f, ax = plt.subplots(nrows=1, ncols=5, figsize=(10, 13))

    ##GR - PE
    ax[0].plot(X[GR],X.index,color='k')

    #GR Fill
    GR_fill(ax[0],X[GR],X.index)

    #PE
    ax1_2 = ax[0].twiny()
    ax1_2.plot(X[PE], X.index,color='blue',alpha=0.5)
    f.text(0.2,0.91,"GR", ha="left", va="bottom", size="medium",color="black")
    f.text(0.2,0.90,"PE", ha="left", va="bottom", size="medium",color="blue",alpha=0.5)

    ##Resistivity
    ax[1].plot(X[MSFL], X.index,color='lightgrey')
    ax[1].plot(X[LLS], X.index,color='grey')
    ax[1].plot(X[LLD], X.index,color='k')
    ax[1].set_xscale("log",
                     #nonposx='clip'
                     ) #log scale
    f.text(0.35,0.92, "MSFL", ha="left", va="bottom", size="medium",color="lightgrey")
    f.text(0.35,0.91, "LLS", ha="left", va="bottom", size="medium",color="grey")
    f.text(0.35,0.90, "LLD", ha="left", va="bottom", size="medium",color="k")

    ##Porosity
    ax[2].plot(X[NPHI], X.index,color='blue')
    ax[2].plot(X[DPHI], X.index,color='red')
    f.text(0.5,0.91,"NPHI", ha="left", va="bottom", size="medium",color="blue")
    f.text(0.5,0.90,"DPHI", ha="left", va="bottom", size="medium",color="red")

    ##Euclidean distance to cluster centers
    ax[3].plot(X[EucD], X.index, '-', color='r')
    f.text(0.65,0.90,"EucD", ha="left", va="bottom", size="medium",color="k")

    #Facies
    #Assigning the colors to the cluster just for this example
    colorlist = []
    for facies in sorted(X.FACIES.unique().tolist()):
        if X['PE'][X.FACIES==facies].mean() > 4:
            colorlist.append('deepskyblue')

        elif 100 > X['GR'][X.FACIES==facies].mean() > 65:
            colorlist.append('peru')

        elif X['GR'][X.FACIES==facies].mean() > 100:
            colorlist.append('dimgrey')
        else:
            colorlist.append('gold')
    cmap = mpl.colors.ListedColormap(colorlist, name='from_list', N=None)#plt.get_cmap('Dark2')
    im=ax[4].imshow(cluster, interpolation='none', aspect='auto',
                    cmap=cmap,vmin=0,vmax=4)
    f.text(0.8,0.90,"FACIES", ha="left", va="bottom", size="medium",color="k")

    #Set repeat tasks with loop
    for i in range(len(ax)-1):
        ax[i].set_ylim(ztop,zbot)
        ax[i].invert_yaxis()
        ax[i].grid()
        try:
            ax[i].locator_params(axis='x', nbins=3) #doesn't seem to like por... hmmm
        except: pass

    ax[0].set_ylabel('MD [ft]', fontsize=16)
    ax[0].grid(False)
    ax[1].grid(False)
    ax[2].grid(False)
    ax[3].grid(False)
    ax[1].set_yticklabels([]); ax[2].set_yticklabels([]); ax[3].set_yticklabels([])
    ax[4].set_yticklabels([]);
    ax[4].set_xticklabels([])

log_plot(X,'GR','MSFL','LLS','LLD','DPHI','NPHI','PE','FACIES','euc_dis')

- Where do you see limestones?
- Where do you see mudstones?
- Where do you see sandstones?
- Where do you see potential source rocks?
- How do those compare to the cluster families?
- Are there unique beds that don't fit their facies well?

---

# Give It A Try

Retrain the K-Means model with different numbers of clusters and plot your results. What zones do you see diffrences? What zones stay the same?  Bonus round: Try to train the model with spectral gamma and compare your results.